# <font color="#418FDE" size="6.5" uppercase>**Advanced Pipelines**</font>

>Last update: 20260126.
    
By the end of this Lecture, you will be able to:
- Optimize tf.data pipelines using caching, interleave, and parallelism settings for large datasets. 
- Configure input pipelines that work efficiently with distributed training strategies. 
- Diagnose and resolve common tf.data performance issues such as slow startup or CPU bottlenecks. 


## **1. Interleave and Cache**

### **1.1. Efficient File Interleaving**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_01_01.jpg?v=1769404835" width="250">



>* Interleaving reads multiple files concurrently to hide latency
>* This keeps batches flowing and hardware fully utilized

>* Balance number of files and prefetch depth
>* Tune interleaving empirically to avoid stalls, overload

>* Use medium-sized shards to avoid file overhead
>* Interleave shards to balance load and improve mixing



In [ ]:
#@title Python Code - Efficient File Interleaving

# This script demonstrates efficient file interleaving.
# We compare sequential and interleaved tf.data pipelines.
# Focus on throughput not model training details.

# !pip install tensorflow==2.20.0.

# Import required TensorFlow and system modules.
import os
import time
import tempfile
import tensorflow as tf

# Set a global random seed for deterministic behavior.
tf.random.set_seed(7)

# Print TensorFlow version in one concise line.
print("TensorFlow version:", tf.__version__)

# Create a temporary directory for our tiny dataset.
base_dir = tempfile.mkdtemp(prefix="interleave_demo_")

# Define simple helper to create small text files.
def create_text_file(path, num_lines, delay_factor):
    with tf.io.gfile.GFile(path, "w") as f:
        for i in range(num_lines):
            f.write(f"{delay_factor},{i}\n")

# Generate a few small files with different artificial delays.
num_files = 4
lines_per_file = 20

# Store file paths for later dataset creation.
file_paths = []
for idx in range(num_files):
    file_path = os.path.join(base_dir, f"file_{idx}.txt")
    create_text_file(file_path, lines_per_file, idx + 1)
    file_paths.append(file_path)

# Convert Python list of paths into a TensorFlow constant.
file_paths_tensor = tf.constant(file_paths)

# Define a parser that simulates slow and fast files.
def parse_line_with_delay(line):
    parts = tf.strings.split(line, ",")
    delay_factor = tf.strings.to_number(parts[0], tf.float32)
    value = tf.strings.to_number(parts[1], tf.float32)
    busy_wait_steps = tf.cast(delay_factor * 2000, tf.int32)
    _ = tf.range(busy_wait_steps)
    return value

# Build a sequential pipeline reading one file at a time.
def build_sequential_dataset():
    ds_files = tf.data.Dataset.from_tensor_slices(file_paths_tensor)
    ds_lines = ds_files.flat_map(
        lambda path: tf.data.TextLineDataset(path)
    )
    ds_values = ds_lines.map(parse_line_with_delay, num_parallel_calls=1)
    ds_values = ds_values.batch(16).prefetch(1)
    return ds_values

# Build an interleaved pipeline reading several files concurrently.
def build_interleaved_dataset():
    ds_files = tf.data.Dataset.from_tensor_slices(file_paths_tensor)
    ds_interleaved = ds_files.interleave(
        lambda path: tf.data.TextLineDataset(path),
        cycle_length=4,
        block_length=4,
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    ds_values = ds_interleaved.map(
        parse_line_with_delay,
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    ds_values = ds_values.cache().batch(16).prefetch(1)
    return ds_values

# Utility to time how long it takes to iterate a dataset.
def time_dataset(dataset, label):
    start = time.time()
    total_batches = 0
    total_elements = 0
    for batch in dataset:
        total_batches += 1
        total_elements += int(batch.shape[0])
    duration = time.time() - start
    print(
        f"{label}: {total_batches} batches, {total_elements} elements, {duration:.3f}s",
    )

# Build both datasets and validate shapes before timing.
sequential_ds = build_sequential_dataset()
interleaved_ds = build_interleaved_dataset()

# Take one batch from each dataset to confirm shapes.
first_seq_batch = next(iter(sequential_ds))
first_int_batch = next(iter(interleaved_ds))
print("Sequential batch shape:", first_seq_batch.shape)
print("Interleaved batch shape:", first_int_batch.shape)

# Time full pass through each dataset to compare throughput.
time_dataset(sequential_ds, "Sequential pipeline")
time_dataset(interleaved_ds, "Interleaved cached pipeline")




### **1.2. Caching Memory vs Disk**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_01_02.jpg?v=1769404919" width="250">



>* In-memory caching speeds up repeated dataset passes
>* Must balance speed gains against limited RAM resources

>* Disk caching is slower than RAM, but scalable
>* Stores preprocessed data snapshot reusable across runs

>* Choose cache type based on measured bottlenecks
>* Combine memory, disk, and selective caching over time



In [ ]:
#@title Python Code - Caching Memory vs Disk

# This script compares memory and disk caching.
# It uses a tiny synthetic image dataset.
# Focus on tf data cache performance basics.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import time
import pathlib

# Import tensorflow and check version.
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Set deterministic random seeds.
SEED = 42
tf.random.set_seed(SEED)

# Define small synthetic image dataset size.
NUM_IMAGES = 256
IMAGE_SHAPE = (28, 28, 1)

# Create a function to generate synthetic images.
def make_synthetic_dataset(num_images, image_shape):
    images = tf.random.uniform(
        shape=(num_images,) + image_shape,
        minval=0.0,
        maxval=1.0,
        dtype=tf.float32,
        seed=SEED,
    )

    labels = tf.zeros((num_images,), dtype=tf.int32)

    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    return ds

# Create base dataset once.
base_ds = make_synthetic_dataset(NUM_IMAGES, IMAGE_SHAPE)

# Define a simple preprocessing function.
@tf.function
def preprocess(image, label):
    image = tf.image.flip_left_right(image)
    image = tf.image.random_brightness(image, 0.1, seed=SEED)
    image = tf.image.per_image_standardization(image)
    return image, label

# Wrap preprocessing for tf data map.
def map_preprocess(image, label):
    image, label = preprocess(image, label)
    return image, label

# Define a helper to build a pipeline.
def build_pipeline(cache_path=None, use_memory=True):
    ds = base_ds.shuffle(NUM_IMAGES, seed=SEED)
    ds = ds.map(map_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if use_memory:
        ds = ds.cache()
    else:
        ds = ds.cache(cache_path)

    ds = ds.batch(32)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

# Define a helper to time one full epoch.
def time_one_epoch(dataset, label):
    start = time.time()
    num_batches = 0
    for batch in dataset:
        num_batches += 1
    end = time.time()
    print(label, "batches:", num_batches, "time:", round(end - start, 3))

# Prepare a temporary cache directory path.
cache_dir = pathlib.Path("./tf_cache_example")
cache_dir.mkdir(exist_ok=True)
cache_file = str(cache_dir / "synthetic_cache.tfdata")

# Build memory cached pipeline.
memory_ds = build_pipeline(use_memory=True)

# Build disk cached pipeline.
disk_ds = build_pipeline(cache_path=cache_file, use_memory=False)

# Time first and second epoch for memory cache.
print("\nMemory cache first epoch (fills cache):")
time_one_epoch(memory_ds, "memory_epoch1")

print("Memory cache second epoch (uses RAM):")
time_one_epoch(memory_ds, "memory_epoch2")

# Time first and second epoch for disk cache.
print("\nDisk cache first epoch (writes cache file):")
time_one_epoch(disk_ds, "disk_epoch1")

print("Disk cache second epoch (reads from disk):")
time_one_epoch(disk_ds, "disk_epoch2")

# Clean up small cache file if it exists.
if os.path.exists(cache_file):
    os.remove(cache_file)



### **1.3. Randomness Performance Tradeoffs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_01_03.jpg?v=1769404959" width="250">



>* Interleaving and parallel reads increase data randomness
>* Caching too early freezes order, hurting generalization

>* Stronger randomness increases CPU, memory, and I O
>* Tuning caching and shuffling balances speed and diversity

>* Place caching and interleaving carefully to balance randomness
>* Start with high randomness, then tune for speed



In [ ]:
#@title Python Code - Randomness Performance Tradeoffs

# This script compares randomness and performance tradeoffs.
# It uses a tiny synthetic dataset with tf.data pipelines.
# Focus on interleave shuffle cache and timing behavior.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import time
import random

# Import tensorflow and check version.
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Set deterministic seeds for reproducibility.
SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

# Create a small synthetic dataset of file identifiers.
num_files = 6
examples_per_file = 20

# Build a list of fake file ids.
file_ids = list(range(num_files))
print("Number of fake files:", len(file_ids))

# Define a function that simulates reading a file.
def read_fake_file(file_id):
    # Create a range of example indices per file.
    ds = tf.data.Dataset.range(examples_per_file)
    # Tag each example with its file id.
    ds = ds.map(lambda x: (file_id, x), num_parallel_calls=1)
    return ds

# Helper to build a pipeline with given options.
def make_pipeline(shuffle_files, interleave_parallel, cache_before_shuffle):
    # Start from dataset of file ids.
    ds = tf.data.Dataset.from_tensor_slices(file_ids)
    # Optionally shuffle file order.
    if shuffle_files:
        ds = ds.shuffle(buffer_size=num_files, seed=SEED)

    # Interleave examples from multiple files.
    ds = ds.interleave(
        lambda fid: read_fake_file(fid),
        cycle_length=interleave_parallel,
        num_parallel_calls=interleave_parallel,
        deterministic=True,
    )

    # Optionally cache before shuffling examples.
    if cache_before_shuffle:
        ds = ds.cache()

    # Shuffle individual examples for randomness.
    ds = ds.shuffle(
        buffer_size=num_files * examples_per_file,
        seed=SEED,
        reshuffle_each_iteration=True,
    )

    # Batch a few examples for inspection.
    ds = ds.batch(8, drop_remainder=False)
    return ds

# Helper to run one epoch and measure time.
def run_epoch(ds, label, max_batches=5):
    # Record start time for this configuration.
    start = time.time()
    first_batches = []

    # Iterate over a few batches only.
    for batch_index, batch in enumerate(ds):
        if batch_index >= max_batches:
            break
        first_batches.append(batch[0])

    # Compute elapsed time in milliseconds.
    elapsed = (time.time() - start) * 1000.0

    # Print summary with first batch source files.
    file_ids_batch = [int(b[0].numpy()) for b in first_batches]
    print(label, "time_ms=", round(elapsed, 2), "first_files=", file_ids_batch)

# Build three pipelines with different tradeoffs.
pipe_strong_random = make_pipeline(
    shuffle_files=True,
    interleave_parallel=4,
    cache_before_shuffle=False,
)

pipe_cached_early = make_pipeline(
    shuffle_files=True,
    interleave_parallel=4,
    cache_before_shuffle=True,
)

pipe_low_parallel = make_pipeline(
    shuffle_files=False,
    interleave_parallel=1,
    cache_before_shuffle=False,
)

# Run two epochs to compare randomness and speed.
for epoch in range(2):
    print("\nEpoch", epoch)
    run_epoch(pipe_strong_random, "strong_random", max_batches=3)
    run_epoch(pipe_cached_early, "cached_early", max_batches=3)
    run_epoch(pipe_low_parallel, "low_parallel", max_batches=3)



## **2. Parallel Input Optimization**

### **2.1. Automatic Parallel Calls**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_02_01.jpg?v=1769405007" width="250">



>* Automatically runs preprocessing on many elements concurrently
>* Keeps multiple devices busy by overlapping data preparation

>* Balances throughput and resource use across replicas
>* Adapts parallelism to CPU capacity and workload

>* Same pipeline scales automatically from single to multi-device
>* System tunes parallelism, improving utilization and consistency



In [ ]:
#@title Python Code - Automatic Parallel Calls

# This script shows automatic parallel calls simply.
# We build a small tf.data pipeline for images.
# Then we compare auto and manual parallel calls.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import random
import time

# Set deterministic seeds for reproducibility.
import numpy as np
import tensorflow as tf

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Detect available devices for context.
physical_gpus = tf.config.list_physical_devices("GPU")
print("GPUs available:", len(physical_gpus))

# Create a simple distributed strategy safely.
if len(physical_gpus) > 1:
    strategy = tf.distribute.MirroredStrategy()
else:
    strategy = tf.distribute.get_strategy()

# Set global random seeds deterministically.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Load MNIST dataset using Keras helper.
(mnist_x_train, mnist_y_train), _ = tf.keras.datasets.mnist.load_data()

# Select a small subset for quick runtime.
subset_size = 4096
mnist_x_train = mnist_x_train[:subset_size]
mnist_y_train = mnist_y_train[:subset_size]

# Validate shapes before building dataset.
assert mnist_x_train.shape[0] == mnist_y_train.shape[0]

# Define a simple preprocessing function.


def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, axis=-1)
    image = tf.image.resize(image, (32, 32))
    image = tf.image.random_flip_left_right(image, seed=seed_value)
    return image, label

# Build a base dataset from tensors.
base_ds = tf.data.Dataset.from_tensor_slices((mnist_x_train, mnist_y_train))

# Shuffle and batch the dataset elements.
base_ds = base_ds.shuffle(buffer_size=subset_size, seed=seed_value)
base_ds = base_ds.batch(64)

# Cache to avoid repeated decoding work.
base_ds = base_ds.cache()

# Create dataset with automatic parallel calls.
auto_ds = base_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
auto_ds = auto_ds.prefetch(tf.data.AUTOTUNE)

# Create dataset with manual single parallel call.
manual_ds = base_ds.map(preprocess, num_parallel_calls=1)
manual_ds = manual_ds.prefetch(1)

# Define a simple model inside strategy scope.
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(32, 32, 1)),
        tf.keras.layers.Conv2D(16, 3, activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax"),
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

# Function to time one short training run.


def time_training(dataset, description):
    start = time.time()
    history = model.fit(
        dataset,
        epochs=1,
        steps_per_epoch=20,
        verbose=0,
    )
    end = time.time()
    duration = end - start
    final_acc = history.history["accuracy"][0]
    print(description, "time:", round(duration, 3), "sec",
          "acc:", round(float(final_acc), 3))

# Warm up model once to stabilize timing.
_ = model.fit(auto_ds.take(5), epochs=1, verbose=0)

# Time training with manual parallel calls.
time_training(manual_ds, "Manual parallel calls")

# Time training with automatic parallel calls.
time_training(auto_ds, "Automatic parallel calls")

# Show one batch shape to confirm pipeline.
for batch_images, batch_labels in auto_ds.take(1):
    print("Batch images shape:", batch_images.shape,
          "Batch labels shape:", batch_labels.shape)




### **2.2. Parallel Mapping Strategies**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_02_02.jpg?v=1769405104" width="250">



>* Parallel mapping spreads heavy preprocessing across resources
>* Proper parallelism keeps GPUs TPUs busy continuously

>* Match parallelism to preprocessing cost and variability
>* Scale parallel mapping with devices to meet demand

>* Balance parallel mapping with other pipeline stages
>* Profile workloads and tune parallelism for each system



In [ ]:
#@title Python Code - Parallel Mapping Strategies

# This script shows parallel mapping strategies.
# It compares sequential and parallel map performance.
# It uses a tiny synthetic image dataset.

# !pip install tensorflow==2.20.0.

# Import required libraries safely.
import os
import time
import random

# Import tensorflow and check version.
import tensorflow as tf

# Set global random seeds for determinism.
SEED_VALUE = 42
random.seed(SEED_VALUE)

# Set numpy and tensorflow seeds deterministically.
import numpy as np
np.random.seed(SEED_VALUE)

tf.random.set_seed(SEED_VALUE)

# Print tensorflow version in one short line.
print("TensorFlow version:", tf.__version__)

# Detect available devices for brief context.
physical_gpus = tf.config.list_physical_devices("GPU")

# Print a compact device summary line.
print("GPUs available:", len(physical_gpus))

# Define tiny synthetic image dataset parameters.
NUM_IMAGES = 2048
IMAGE_HEIGHT = 28

# Define remaining image shape parameters.
IMAGE_WIDTH = 28
NUM_CHANNELS = 1

# Create small random image array deterministically.
images = np.random.randint(
    0,
    256,
    size=(NUM_IMAGES, IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS),
    dtype=np.uint8,
)

# Create simple labels for completeness.
labels = np.random.randint(
    0,
    10,
    size=(NUM_IMAGES,),
    dtype=np.int32,
)

# Wrap numpy arrays into a tf.data dataset.
base_ds = tf.data.Dataset.from_tensor_slices((images, labels))

# Shuffle and batch to mimic training input.
base_ds = base_ds.shuffle(NUM_IMAGES, seed=SEED_VALUE)

# Define global batch size for distributed training.
GLOBAL_BATCH_SIZE = 128

# Batch dataset with drop remainder for stability.
base_ds = base_ds.batch(GLOBAL_BATCH_SIZE, drop_remainder=True)

# Define an expensive preprocessing function.
def heavy_preprocess(image, label):
    # Cast image to float and normalize.
    image = tf.cast(image, tf.float32) / 255.0

    # Apply random flip for augmentation.
    image = tf.image.random_flip_left_right(image, seed=SEED_VALUE)

    # Apply random brightness jitter.
    image = tf.image.random_brightness(image, max_delta=0.1)

    # Simulate extra CPU work with convolution.
    kernel = tf.ones((3, 3, NUM_CHANNELS, NUM_CHANNELS)) / 9.0

    image = tf.nn.conv2d(
        image,
        kernel,
        strides=1,
        padding="SAME",
    )

    # Return processed image and original label.
    return image, label

# Build dataset with sequential mapping configuration.
seq_ds = base_ds.map(
    heavy_preprocess,
    num_parallel_calls=1,
)

# Prefetch to overlap mapping and training.
seq_ds = seq_ds.prefetch(tf.data.AUTOTUNE)

# Build dataset with parallel mapping configuration.
par_ds = base_ds.map(
    heavy_preprocess,
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Prefetch for the parallel dataset as well.
par_ds = par_ds.prefetch(tf.data.AUTOTUNE)

# Define a simple function to time one epoch.
def time_one_epoch(dataset, num_steps):
    # Record start time using time module.
    start = time.time()

    # Iterate fixed number of steps defensively.
    step_count = 0

    for batch_images, batch_labels in dataset:
        # Validate batch shapes before using.
        if batch_images.shape[0] != GLOBAL_BATCH_SIZE:
            break

        # Simple lightweight computation per batch.
        _ = tf.reduce_mean(batch_images)

        step_count += 1

        if step_count >= num_steps:
            break

    # Compute elapsed time in seconds.
    elapsed = time.time() - start

    # Return elapsed time and actual steps.
    return elapsed, step_count

# Determine safe number of steps for timing.
TOTAL_STEPS = int(NUM_IMAGES / GLOBAL_BATCH_SIZE)

# Limit steps to keep runtime very small.
MEASURE_STEPS = min(TOTAL_STEPS, 10)

# Warm up both datasets briefly before timing.
_ = next(iter(seq_ds))
_ = next(iter(par_ds))

# Time sequential mapping dataset throughput.
seq_time, seq_steps = time_one_epoch(seq_ds, MEASURE_STEPS)

# Time parallel mapping dataset throughput.
par_time, par_steps = time_one_epoch(par_ds, MEASURE_STEPS)

# Compute images per second for sequential mapping.
seq_images_per_sec = (seq_steps * GLOBAL_BATCH_SIZE) / max(seq_time, 1e-6)

# Compute images per second for parallel mapping.
par_images_per_sec = (par_steps * GLOBAL_BATCH_SIZE) / max(par_time, 1e-6)

# Print concise comparison of both strategies.
print("Sequential map steps:", seq_steps, "time:", round(seq_time, 3))

# Print parallel mapping timing information.
print("Parallel map steps:", par_steps, "time:", round(par_time, 3))

# Print throughput numbers for both configurations.
print("Sequential images/sec:", int(seq_images_per_sec))

# Print parallel throughput to highlight improvement.
print("Parallel images/sec:", int(par_images_per_sec))

# Show simple speedup factor for quick intuition.
print("Parallel speedup factor:", round(par_images_per_sec / seq_images_per_sec, 2))




### **2.3. Accelerating GPU and TPU**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_02_03.jpg?v=1769405145" width="250">



>* Keep GPUs and TPUs constantly fed with data
>* Overlap host preprocessing with training across all replicas

>* Match parallelism and prefetching to replica demand
>* Use stronger host side pipelines, especially on TPUs

>* Slow input pipelines leave accelerators idle, underused
>* Profile and tune parallelism to unlock linear scaling



In [ ]:
#@title Python Code - Accelerating GPU and TPU

# This script shows parallel input optimization.
# We focus on accelerators and distributed strategies.
# All examples are small and beginner friendly.

# # Uncomment if TensorFlow is not already installed.
# !pip install -q tensorflow==2.20.0.

# Import required modules for TensorFlow usage.
import os
import random
import numpy as np
import tensorflow as tf

# Set deterministic seeds for reproducible behavior.
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Detect available devices and possible accelerators.
physical_gpus = tf.config.list_physical_devices("GPU")
physical_tpus = tf.config.list_physical_devices("TPU")
print("GPUs:", len(physical_gpus), "TPUs:", len(physical_tpus))

# Choose a distribution strategy for demonstration.
if physical_tpus:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(resolver)
elif physical_gpus:
    strategy = tf.distribute.MirroredStrategy()
else:
    strategy = tf.distribute.OneDeviceStrategy("/cpu:0")

# Report the number of replicas in the strategy.
num_replicas = strategy.num_replicas_in_sync
print("Replicas in sync:", num_replicas)

# Load a small subset of MNIST image data.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
subset_size = 6000
x_train = x_train[:subset_size]
y_train = y_train[:subset_size]

# Normalize and add channel dimension for images.
x_train = x_train.astype("float32") / 255.0
x_train = np.expand_dims(x_train, axis=-1)
print("Train subset shape:", x_train.shape)

# Define a simple preprocessing function for dataset.
def preprocess(image, label):
    image = tf.image.resize(image, (28, 28))
    image = tf.image.random_flip_left_right(image)
    return image, label

# Create a base tf.data dataset from numpy arrays.
base_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
base_ds = base_ds.shuffle(buffer_size=subset_size, seed=seed_value)

# Compute global and per replica batch sizes.
global_batch_size = 128
per_replica_batch = global_batch_size // num_replicas
if global_batch_size % num_replicas != 0:
    per_replica_batch = max(1, per_replica_batch)

# Build an optimized pipeline with parallel mapping.
num_parallel_calls = tf.data.AUTOTUNE
optimized_ds = base_ds.map(preprocess, num_parallel_calls=num_parallel_calls)
optimized_ds = optimized_ds.cache()

# Use interleave to simulate multiple file shards.
optimized_ds = optimized_ds.interleave(
    lambda x, y: tf.data.Dataset.from_tensors((x, y)),
    cycle_length=4,
    num_parallel_calls=num_parallel_calls,
)

# Batch and prefetch to overlap host and device work.
optimized_ds = optimized_ds.batch(global_batch_size, drop_remainder=True)
optimized_ds = optimized_ds.prefetch(tf.data.AUTOTUNE)

# Distribute the dataset for the chosen strategy.
dist_dataset = strategy.experimental_distribute_dataset(optimized_ds)

# Define a very small convolutional model.
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(10, activation="softmax"),
    ])
    return model

# Create and compile the model inside strategy scope.
with strategy.scope():
    model = create_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )

# Train briefly to show pipeline and accelerator usage.
history = model.fit(dist_dataset, epochs=1, steps_per_epoch=20, verbose=0)

# Print final training metrics from the short run.
final_loss = history.history["loss"][-1]
final_acc = history.history["accuracy"][-1]
print("Final loss:", round(float(final_loss), 4))
print("Final accuracy:", round(float(final_acc), 4))




## **3. Debugging Data Pipelines**

### **3.1. Quick Dataset Inspection**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_03_01.jpg?v=1769405184" width="250">



>* Quickly iterate a few batches, checking outputs
>* Watch startup delays to spot expensive initialization work

>* Inspect element structure to avoid unnecessary work
>* Simplify data formats and precompute repeated metadata

>* Time a few batches versus training step
>* Use timing patterns to locate pipeline bottlenecks



In [ ]:
#@title Python Code - Quick Dataset Inspection

# This script inspects a tf.data pipeline quickly.
# It focuses on timing and element structure.
# Use it to spot simple performance issues.

# !pip install tensorflow==2.20.0.

# Import required standard libraries.
import os
import time
import random

# Set deterministic random seeds.
random.seed(7)
os.environ["PYTHONHASHSEED"] = "7"

# Import TensorFlow and check version.
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Create a small synthetic image dataset.
num_samples = 64
image_height = 28

# Validate simple positive sizes.
assert num_samples > 0 and image_height > 0

# Build random image and label tensors.
images = tf.random.uniform(
    shape=(num_samples, image_height, image_height, 1)
)
labels = tf.random.uniform(
    shape=(num_samples,), maxval=10, dtype=tf.int32
)

# Wrap tensors in a tf.data Dataset.
base_ds = tf.data.Dataset.from_tensor_slices((images, labels))

# Define a simple preprocessing function.
def preprocess(example_image, example_label):
    image = tf.cast(example_image, tf.float32) / 255.0
    label = tf.cast(example_label, tf.int32)
    return image, label


# Apply preprocessing and batching.
batch_size = 8
inspected_ds = base_ds.map(preprocess).batch(batch_size)

# Take a few batches for quick inspection.
num_inspect_batches = 3

# Time how long it takes to get batches.
start_time = time.time()
first_batch_time = None

# Iterate over a small number of batches.
for batch_index, batch in enumerate(inspected_ds.take(num_inspect_batches)):
    batch_images, batch_labels = batch
    now = time.time()
    if batch_index == 0:
        first_batch_time = now - start_time
    print(
        "Batch", batch_index,
        "shape", batch_images.shape,
        "dtype", batch_images.dtype
    )
    print(
        "Labels shape", batch_labels.shape,
        "dtype", batch_labels.dtype
    )

# Compute total elapsed time for inspected batches.
total_time = time.time() - start_time

# Print simple timing summary for inspection.
print("First batch seconds:", round(first_batch_time, 4))
print("Total seconds for batches:", round(total_time, 4))
print("Average seconds per batch:", round(total_time / num_inspect_batches, 4))



### **3.2. Profiling Input Performance**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_03_02.jpg?v=1769405239" width="250">



>* Treat the input pipeline as measurable system
>* Compare startup behavior versus steady-state throughput patterns

>* Time isolated pipelines over fixed batch counts
>* Use timings to locate bottlenecks and startup overheads

>* Track CPU, GPU, I O, and prefetch overlap
>* Correlate timings to confirm real bottleneck improvements



In [ ]:
#@title Python Code - Profiling Input Performance

# This script profiles tf.data input performance simply.
# It compares slow and optimized pipelines for beginners.
# Focus on timing batches and checking device utilization.

# !pip install tensorflow==2.20.0.

# Import required standard libraries safely.
import os
import time
import random

# Import TensorFlow and check version.
import tensorflow as tf

# Set deterministic seeds for reproducibility.
random.seed(7)

# Set TensorFlow random seed deterministically.
tf.random.set_seed(7)

# Print TensorFlow version in one concise line.
print("TensorFlow version:", tf.__version__)

# Detect available physical devices for context.
physical_devices = tf.config.list_physical_devices()

# Print number of detected devices briefly.
print("Detected devices:", len(physical_devices))

# Create a small synthetic dataset for profiling.
num_examples = 2000

# Build a simple tensor of integer ids.
ids = tf.range(num_examples, dtype=tf.int32)

# Define a function simulating expensive preprocessing.
def slow_preprocess(x: tf.Tensor) -> tf.Tensor:
    x = tf.cast(x, tf.float32)
    for _ in range(5):
        x = tf.math.sqrt(x + 1.0)
    return x

# Define a function simulating lighter preprocessing.
def fast_preprocess(x: tf.Tensor) -> tf.Tensor:
    x = tf.cast(x, tf.float32)
    return x * 0.5 + 1.0

# Helper function to time iteration over some batches.
def time_dataset(ds: tf.data.Dataset, num_batches: int) -> float:
    start = time.time()
    count = 0
    for batch in ds.take(num_batches):
        _ = tf.reduce_sum(batch)
        count += 1
    end = time.time()
    if count == 0:
        return 0.0
    return (end - start) / float(count)

# Build a deliberately slow input pipeline.
slow_ds = tf.data.Dataset.from_tensor_slices(ids)

# Add slow mapping without parallel calls.
slow_ds = slow_ds.map(slow_preprocess)

# Batch the slow dataset with small batch size.
slow_ds = slow_ds.batch(32)

# Prefetch minimally to show limited overlap.
slow_ds = slow_ds.prefetch(1)

# Build an optimized pipeline using parallelism.
opt_ds = tf.data.Dataset.from_tensor_slices(ids)

# Map with fast preprocessing and autotune parallelism.
opt_ds = opt_ds.map(
    fast_preprocess,
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Batch with same size for fair comparison.
opt_ds = opt_ds.batch(32)

# Prefetch aggressively using autotune setting.
opt_ds = opt_ds.prefetch(tf.data.AUTOTUNE)

# Warm up both datasets to reduce startup noise.
_ = next(iter(slow_ds.take(1)))

# Warm up optimized dataset similarly.
_ = next(iter(opt_ds.take(1)))

# Choose number of batches for timing runs.
num_timing_batches = 50

# Time the slow pipeline average batch latency.
slow_latency = time_dataset(slow_ds, num_timing_batches)

# Time the optimized pipeline average batch latency.
opt_latency = time_dataset(opt_ds, num_timing_batches)

# Compute examples per second for slow pipeline.
slow_eps = 32.0 / slow_latency if slow_latency > 0 else 0.0

# Compute examples per second for optimized pipeline.
opt_eps = 32.0 / opt_latency if opt_latency > 0 else 0.0

# Print concise timing summary for both pipelines.
print("Slow pipeline avg batch seconds:", round(slow_latency, 4))

# Print optimized pipeline timing for comparison.
print("Optimized pipeline avg batch seconds:", round(opt_latency, 4))

# Print throughput in examples per second values.
print("Slow pipeline examples per second:", int(slow_eps))

# Print optimized throughput for quick inspection.
print("Optimized pipeline examples per second:", int(opt_eps))

# Build a tiny model to observe input interaction.
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)),
    tf.keras.layers.Dense(8, activation="relu"),
    tf.keras.layers.Dense(1),
])

# Compile model with simple optimizer and loss.
model.compile(optimizer="adam", loss="mse")

# Create labels tensor matching dataset batches.
labels = tf.cast(ids[:num_examples], tf.float32)

# Zip features and labels for training dataset.
train_ds = tf.data.Dataset.from_tensor_slices((ids, labels))

# Apply slow preprocessing only to features here.
train_slow = train_ds.map(
    lambda x, y: (slow_preprocess(x), y),
)

# Batch and prefetch slow training dataset.
train_slow = train_slow.batch(32).prefetch(1)

# Apply fast preprocessing for optimized training dataset.
train_opt = train_ds.map(
    lambda x, y: (fast_preprocess(x), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)

# Batch and prefetch optimized training dataset.
train_opt = train_opt.batch(32).prefetch(tf.data.AUTOTUNE)

# Train briefly on slow pipeline and time steps.
start_slow_train = time.time()
model.fit(train_slow, epochs=1, verbose=0)
end_slow_train = time.time()

# Train briefly on optimized pipeline and time steps.
start_opt_train = time.time()
model.fit(train_opt, epochs=1, verbose=0)
end_opt_train = time.time()

# Compute per epoch durations for both pipelines.
slow_epoch_time = end_slow_train - start_slow_train

# Compute optimized epoch duration similarly.
opt_epoch_time = end_opt_train - start_opt_train

# Print concise training time comparison summary.
print("Slow pipeline epoch seconds:", round(slow_epoch_time, 3))



### **3.3. Handling data order determinism**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_05/Lecture_B/image_03_03.jpg?v=1769405356" width="250">



>* Deterministic order aids reproducibility and debugging reliability
>* Relaxed order boosts parallelism and overall throughput

>* Strict ordering can stall pipelines and GPUs
>* Relaxed determinism boosts parallelism, hiding slow reads

>* Choose deterministic mode for debugging and comparisons
>* Switch to non-deterministic mode for faster training



In [ ]:
#@title Python Code - Handling data order determinism

# This script shows deterministic versus nondeterministic pipelines.
# It focuses on tf.data order and performance tradeoffs.
# Run cells to compare behavior and printed element orders.

# !pip install tensorflow==2.20.0.

# Import required modules safely.
import os
import time
import random

# Set deterministic seeds for reproducibility.
import numpy as np
import tensorflow as tf

# Print TensorFlow version briefly.
print("TensorFlow version:", tf.__version__)

# Create a small base dataset of integers.
base_values = tf.range(start=0, limit=8, delta=1)

# Define an artificial slow mapping function.
@tf.function


def slow_square(x):
    # Add tiny sleep using tf.py_function.
    def _sleep_and_square(v):
        time.sleep(0.01 + float(v % 3) * 0.005)
        return np.int32(v * v)

    # Wrap Python function for TensorFlow.
    y = tf.py_function(_sleep_and_square, [x], Tout=tf.int32)
    # Ensure shape information is preserved.
    y.set_shape(x.shape)
    return y

# Build a deterministic pipeline with parallel map.
def make_deterministic_dataset():
    # Start from base tensor dataset.
    ds = tf.data.Dataset.from_tensor_slices(base_values)
    # Map with parallel calls and deterministic order.
    ds = ds.map(
        slow_square,
        num_parallel_calls=tf.data.AUTOTUNE,
        deterministic=True,
    )
    # Batch elements for efficiency.
    ds = ds.batch(4)
    return ds

# Build a nondeterministic pipeline with parallel map.
def make_nondeterministic_dataset():
    # Start from same base tensor dataset.
    ds = tf.data.Dataset.from_tensor_slices(base_values)
    # Map with parallel calls and relaxed order.
    ds = ds.map(
        slow_square,
        num_parallel_calls=tf.data.AUTOTUNE,
        deterministic=False,
    )
    # Batch elements for efficiency.
    ds = ds.batch(4)
    return ds

# Helper to iterate once and collect batches.
def run_once(dataset, label):
    # Record start time for simple timing.
    start = time.time()
    batches = []
    for batch in dataset:
        batches.append(batch.numpy())
    duration = time.time() - start
    # Print label, order, and duration.
    print(label, "batches:", batches)
    print(label, "time_sec:", round(duration, 3))

# Run deterministic pipeline and observe order.
run_once(make_deterministic_dataset(), "deterministic")

# Run nondeterministic pipeline and observe order.
run_once(make_nondeterministic_dataset(), "nondeterministic")

# Run nondeterministic pipeline again to show variability.
run_once(make_nondeterministic_dataset(), "nondeterministic_run2")




# <font color="#418FDE" size="6.5" uppercase>**Advanced Pipelines**</font>


In this lecture, you learned to:
- Optimize tf.data pipelines using caching, interleave, and parallelism settings for large datasets. 
- Configure input pipelines that work efficiently with distributed training strategies. 
- Diagnose and resolve common tf.data performance issues such as slow startup or CPU bottlenecks. 

In the next Module (Module 6), we will go over 'Computer Vision'